In [2]:
import pysal as ps
import numpy as np
import scipy as sp
import pandas as pd
from gwr.sel_bw import Sel_BW
from gwr.gwr import GWR, MGWR
import statsmodels.api as smf
from statsmodels.tools import add_constant

In [3]:
#A single covariate for simulated data

In [4]:
X1 = np.random.normal(0, 1, 625).reshape((-1,1))
X = np.hstack([X1])
B1 = 5
B2 = -2
epsilon = np.random.normal(0,1)
y = X1*B1 + epsilon

In [5]:
smf.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.944
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                 1.050e+04
Date:                Tue, 20 Mar 2018   Prob (F-statistic):               0.00
Time:                        15:50:01   Log-Likelihood:                -1014.2
No. Observations:                 625   AIC:                             2030.
Df Residuals:                     624   BIC:                             2035.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             4.9402      0.048    102.486      0.000       4.846       5.035
==============================================================================
Omnibus:                        0.005   Durbin-Watson:                   0.005
Prob(Omnibus):                  0.998   Jarque-Bera (JB):                0.028
Skew:                           0.006   Prob(JB):                        0.986
Kurtosis:                       2.970   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
x_coord = np.repeat(range(1,26), 25).astype(float)
y_coord = np.tile(range(1,26), 25).astype(float)
coords = zip(x_coord, y_coord)

In [7]:
sel = Sel_BW(coords, y, X, constant=False)

bw = sel.search()
print bw
results = GWR(coords, y, X, bw, constant=False).fit()

624.0


In [8]:
results.params[0:5]

array([[4.93920094],
       [4.93930817],
       [4.93944166],
       [4.9396059 ],
       [4.93980616]])

In [9]:
selector = Sel_BW(coords, y, X, multi=True, constant=False)
bw = selector.search(tol_multi=.00001)
err = selector.err
XB = selector.XB
gwr = MGWR(coords, y, X, bw, XB, err, constant=False).fit()

In [10]:
print gwr.model.bws

[624.0]


In [11]:
gwr.params[0:5]

array([[4.93920094],
       [4.93930817],
       [4.93944166],
       [4.9396059 ],
       [4.93980616]])

In [12]:
#MGWR hat matrix
S = selector.bw[-1]

#lets manually compute sigma**2. We can define it based on H+T or as that from GWR
sigma1 = gwr.resid_ss /np.trace(np.dot(S, S.T))
sigma2 = gwr.resid_ss / (len(gwr.model.y) - (2*np.trace(S)) + np.trace(np.dot(S.T, S)))

In [13]:
#GWR vs MGWR for sigma**2
results.sigma2_v1v2, sigma1, sigma2

(1.5079700659504651, 729.544674360347, 1.5079700659504651)

In [14]:
#GWR standard errors
results.bse[0:5]

array([[0.05593041],
       [0.05584191],
       [0.05577079],
       [0.05572122],
       [0.05569806]])

In [15]:
#MGWR standard errors according to H+T
S_0 = gwr.S[0]
bse = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma1))
bse[0:5].reshape((-1,1))

array([[1.23665961],
       [1.07858322],
       [0.24608715],
       [2.62055707],
       [1.08406403]])

In [16]:
#MGWR standard errors according to H+T but using GWR type DOF
S_0 = gwr.S[0]
bse = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma2))
bse[0:5].reshape((-1,1))

array([[0.05622385],
       [0.04903702],
       [0.01118818],
       [0.11914177],
       [0.0492862 ]])

In [17]:
#MGWR standard errors according to H+T and using Hanchen's suggestion to factor out Xj
X_0 = gwr.model.X[:,0]
S_0 = gwr.S[0]
S_0 = np.linalg.inv(np.diag(X_0)) * S_0


bse = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma1))
bse[0:5].reshape((-1,1))

array([[0.09143401],
       [0.07828983],
       [0.01756454],
       [0.18425109],
       [0.0752289 ]])

In [18]:
#MGWR standard errors according to H+T but using GWR type DOF and using Hanchen's suggestion to factor out Xj
X_0 = gwr.model.X[:,0]
S_0 = gwr.S[0]
S_0 = np.linalg.inv(np.diag(X_0)) * S_0


bse = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma2))
bse[0:5].reshape((-1,1))

array([[0.00415698],
       [0.00355939],
       [0.00079856],
       [0.00837684],
       [0.00342023]])

In [19]:
#MGWR standard errors if we just pull out the individual GWR standard error for the single component
gwr.SE[0][0:5]

array([[0.05593041],
       [0.05584191],
       [0.05577079],
       [0.05572122],
       [0.05569806]])

In [20]:
#Two covariates for simulated data

In [58]:
X1 = np.random.normal(0, 1, 625).reshape((-1,1))
X2 = np.random.normal(0, 1, 625).reshape((-1,1))
X = np.hstack([X1, X2])
B1 = 5
B2 = -2
epsilon = np.random.normal(0,1)
y = X1*B1 + X2*B2 + epsilon

In [59]:
smf.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.982
Method:                 Least Squares   F-statistic:                 1.711e+04
Date:                Tue, 20 Mar 2018   Prob (F-statistic):               0.00
Time:                        11:39:30   Log-Likelihood:                -689.16
No. Observations:                 625   AIC:                             1382.
Df Residuals:                     623   BIC:                             1391.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             5.0089      0.029    171.911      0.000       4.952       5.066
x2            -1.9843      0.028    -71.154      0.000      -2.039      -1.930
==============================================================================
Omnibus:                        0.442   Durbin-Watson:                   0.001
Prob(Omnibus):                  0.802   Jarque-Bera (JB):                0.524
Skew:                           0.056   Prob(JB):                        0.770
Kurtosis:                       2.914   Cond. No.                         1.05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [60]:
x_coord = np.repeat(range(1,26), 25).astype(float)
y_coord = np.tile(range(1,26), 25).astype(float)
coords = zip(x_coord, y_coord)

In [61]:
sel = Sel_BW(coords, y, X, constant=False)

bw = sel.search()
print bw
results = GWR(coords, y, X, bw, constant=False).fit()

621.0


In [62]:
results.params[0:5]

array([[ 5.03392247, -1.96840842],
       [ 5.03467376, -1.96722223],
       [ 5.03537019, -1.96601055],
       [ 5.03607808, -1.96473485],
       [ 5.03679231, -1.96339331]])

In [63]:
selector = Sel_BW(coords, y, X, multi=True, constant=False)
bw = selector.search(tol_multi=.00001)
err = selector.err
XB = selector.XB
gwr = MGWR(coords, y, X, bw, XB, err, constant=False).fit()

In [64]:
print gwr.model.bws

[624.0, 620.0]


In [65]:
gwr.params[0:5]

array([[ 5.03251248, -1.96857167],
       [ 5.03310937, -1.96742374],
       [ 5.03371685, -1.96616986],
       [ 5.03433179, -1.9648413 ],
       [ 5.03494995, -1.96343464]])

In [96]:
#MGWR hat matrix
S = selector.bw[-1]

#lets manually compute sigma**2. We can define it based on H+T or as that from GWR
sigma1 = gwr.resid_ss / np.trace(np.dot(S, S.T))
sigma2 = gwr.resid_ss / (len(gwr.model.y) - (2*np.trace(S)) + np.trace(np.dot(S.T, S)))

In [97]:
#GWR vs MGWR for sigma**2
results.sigma2_v1v2, sigma1, sigma2

(0.5323786548390854, 126.0676413945019, 0.5323753796207696)

In [70]:
#GWR standard errors
results.bse[0:5]

array([[0.03448996, 0.03270217],
       [0.03442739, 0.03265254],
       [0.03435185, 0.03259237],
       [0.03428892, 0.03254319],
       [0.0342421 , 0.03250807]])

In [90]:
#MGWR standard errors according to H+T
S_0 = gwr.S[0]
S_1 = gwr.S[1]
bse0 = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma1)).reshape((-1,1))
bse1 = np.sqrt(np.diag(np.dot(S_1, S_1.T)*sigma1)).reshape((-1,1))

bse = np.hstack([bse0, bse1])

bse[0:5]

array([[1.1036064 , 0.61579235],
       [0.2580416 , 0.10862743],
       [0.68667834, 1.88472969],
       [0.72188349, 0.39252593],
       [0.03292367, 0.28571795]])

In [91]:
#MGWR standard errors according to H+T but using GWR type DOF
S_0 = gwr.S[0]
S_1 = gwr.S[1]
bse0 = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma2)).reshape((-1,1))
bse1 = np.sqrt(np.diag(np.dot(S_1, S_1.T)*sigma2)).reshape((-1,1))

bse = np.hstack([bse0, bse1])

bse[0:5]

array([[0.07171688, 0.04001672],
       [0.0167686 , 0.00705906],
       [0.04462318, 0.12247748],
       [0.04691096, 0.02550795],
       [0.00213952, 0.01856713]])

In [93]:
#MGWR standard errors according to H+T and using Hanchen's suggestion to factor out Xj
X_0 = gwr.model.X[:,0]
S_0 = gwr.S[0]
S_0 = np.linalg.inv(np.diag(X_0)) * S_0
X_1 = gwr.model.X[:,1]
S_1 = gwr.S[1]
S_1 = np.linalg.inv(np.diag(X_1)) * S_1

bse0 = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma1)).reshape((-1,1))
bse1 = np.sqrt(np.diag(np.dot(S_1, S_1.T)*sigma1)).reshape((-1,1))

bse = np.hstack([bse0, bse1])

bse[0:5]

array([[0.08310743, 0.0432959 ],
       [0.01912749, 0.00751512],
       [0.05018995, 0.12865381],
       [0.05212497, 0.02649321],
       [0.00235337, 0.01911187]])

In [94]:
#MGWR standard errors according to H+T but using GWR type DOF and using Hanchen's suggestion to factor out Xj
X_0 = gwr.model.X[:,0]
S_0 = gwr.S[0]
S_0 = np.linalg.inv(np.diag(X_0)) * S_0
X_1 = gwr.model.X[:,1]
S_1 = gwr.S[1]
S_1 = np.linalg.inv(np.diag(X_1)) * S_1

bse0 = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma2)).reshape((-1,1))
bse1 = np.sqrt(np.diag(np.dot(S_1, S_1.T)*sigma2)).reshape((-1,1))

bse = np.hstack([bse0, bse1])

bse[0:5]

array([[0.00540066, 0.00281355],
       [0.00124298, 0.00048836],
       [0.00326155, 0.00836045],
       [0.00338729, 0.00172164],
       [0.00015293, 0.00124197]])

In [95]:
#MGWR standard errors if we just pull out the individual GWR standard error for the single component
np.hstack([gwr.SE[0], gwr.SE[1]])[0:5]

array([[0.03394429, 0.0326568 ],
       [0.03385135, 0.03258816],
       [0.03376745, 0.03254779],
       [0.03369531, 0.03251936],
       [0.03363816, 0.03250609]])

In [98]:
#Now an empirical data example

In [140]:
data = ps.open(ps.examples.get_path('GData_utm.csv'))


y = np.array(data.by_col['PctBach']).reshape((-1,1))
#pov = np.array(data.by_col['PctPov']).reshape((-1,1))
rural = np.array(data.by_col['PctRural']).reshape((-1,1))
#black = np.array(data.by_col['PctBlack']).reshape((-1,1))
fb = np.array(data.by_col['PctFB']).reshape((-1,1))

X = np.hstack([fb, rural])

#Coordinates for calibration points
u = data.by_col['X']
v = data.by_col['Y']
coords = zip(u,v)

In [141]:
smf.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.823
Model:                            OLS   Adj. R-squared:                  0.821
Method:                 Least Squares   F-statistic:                     364.4
Date:                Tue, 20 Mar 2018   Prob (F-statistic):           1.03e-59
Time:                        12:23:15   Log-Likelihood:                -487.50
No. Observations:                 159   AIC:                             979.0
Df Residuals:                     157   BIC:                             985.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             4.5877      0.283     16.183      0.000       4.028       5.148
x2             0.0694      0.006     11.011      0.000       0.057       0.082
==============================================================================
Omnibus:                       20.998   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               78.796
Skew:                          -0.315   Prob(JB):                     7.76e-18
Kurtosis:                       6.391   Cond. No.                         51.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [142]:
sel = Sel_BW(coords, y, X, constant=False)

bw = sel.search()
print bw
results = GWR(coords, y, X, bw, constant=False).fit()

116.0


In [143]:
results.params[0:5]

array([[4.10854216, 0.0686757 ],
       [3.2914029 , 0.07591392],
       [3.86347867, 0.07057871],
       [2.78077223, 0.08451004],
       [5.74472239, 0.06879297]])

In [146]:
selector = Sel_BW(coords, y, X, multi=True, constant=False)
bw = selector.search(bw_min=116, bw_max=116, tol_multi=.00001)
err = selector.err
XB = selector.XB
gwr = MGWR(coords, y, X, bw, XB, err, constant=False).fit()

In [147]:
print gwr.model.bws

[116.0, 116.0]


In [148]:
gwr.params[0:5]

array([[4.03334434, 0.06890593],
       [3.44447653, 0.07016332],
       [3.85225066, 0.06921807],
       [3.0296313 , 0.07483795],
       [5.68267915, 0.07326189]])

In [149]:
#MGWR hat matrix
S = selector.bw[-1]

#lets manually compute sigma**2. We can define it based on H+T or as that from GWR
sigma1 = gwr.resid_ss / np.trace(np.dot(S, S.T))
sigma2 = gwr.resid_ss / (len(gwr.model.y) - (2*np.trace(S)) + np.trace(np.dot(S.T, S)))

In [150]:
#GWR vs MGWR for sigma**2
results.sigma2_v1v2, sigma1, sigma2

(23.228755996597254, 841.2714610691412, 23.394349843221303)

In [151]:
#GWR standard errors
results.bse[0:5]

array([[0.55407423, 0.0094543 ],
       [0.46580558, 0.00873035],
       [0.52377   , 0.00919138],
       [0.42558204, 0.00900781],
       [0.34132921, 0.00832654]])

In [152]:
#MGWR standard errors according to H+T
S_0 = gwr.S[0]
S_1 = gwr.S[1]
bse0 = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma1)).reshape((-1,1))
bse1 = np.sqrt(np.diag(np.dot(S_1, S_1.T)*sigma1)).reshape((-1,1))

bse = np.hstack([bse0, bse1])

bse[0:5]

array([[1.71969794, 3.46334654],
       [3.74559544, 4.42224048],
       [0.69573198, 2.78461077],
       [0.24394397, 4.69218778],
       [2.699523  , 1.95399106]])

In [153]:
#MGWR standard errors according to H+T but using GWR type DOF
S_0 = gwr.S[0]
S_1 = gwr.S[1]
bse0 = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma2)).reshape((-1,1))
bse1 = np.sqrt(np.diag(np.dot(S_1, S_1.T)*sigma2)).reshape((-1,1))

bse = np.hstack([bse0, bse1])

bse[0:5]

array([[0.28677387, 0.57754172],
       [0.62460906, 0.73744523],
       [0.11601907, 0.46435691],
       [0.04067968, 0.78246117],
       [0.45016782, 0.32584419]])

In [154]:
#MGWR standard errors according to H+T and using Hanchen's suggestion to factor out Xj
X_0 = gwr.model.X[:,0]
S_0 = gwr.S[0]
S_0 = np.linalg.inv(np.diag(X_0)) * S_0
X_1 = gwr.model.X[:,1]
S_1 = gwr.S[1]
S_1 = np.linalg.inv(np.diag(X_1)) * S_1

bse0 = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma1)).reshape((-1,1))
bse1 = np.sqrt(np.diag(np.dot(S_1, S_1.T)*sigma1)).reshape((-1,1))

bse = np.hstack([bse0, bse1])

bse[0:5]

array([[0.24091971, 0.00803093],
       [0.49676555, 0.01018216],
       [0.09526546, 0.00638057],
       [0.02577187, 0.01117829],
       [0.36687855, 0.00481712]])

In [155]:
#MGWR standard errors according to H+T but using GWR type DOF and using Hanchen's suggestion to factor out Xj
X_0 = gwr.model.X[:,0]
S_0 = gwr.S[0]
S_0 = np.linalg.inv(np.diag(X_0)) * S_0
X_1 = gwr.model.X[:,1]
S_1 = gwr.S[1]
S_1 = np.linalg.inv(np.diag(X_1)) * S_1

bse0 = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma2)).reshape((-1,1))
bse1 = np.sqrt(np.diag(np.dot(S_1, S_1.T)*sigma2)).reshape((-1,1))

bse = np.hstack([bse0, bse1])

bse[0:5]

array([[0.04017536, 0.00133922],
       [0.08283977, 0.00169796],
       [0.0158863 , 0.00106401],
       [0.00429767, 0.00186407],
       [0.06118004, 0.00080329]])

In [156]:
#MGWR standard errors if we just pull out the individual GWR standard error for the single component
np.hstack([gwr.SE[0], gwr.SE[1]])[0:5]

array([[0.44233896, 0.00754668],
       [0.39025347, 0.00728491],
       [0.42419076, 0.00743466],
       [0.36507357, 0.0077296 ],
       [0.31076635, 0.00753835]])

In [157]:
#Now lets try with adding in a constant

In [240]:
data = ps.open(ps.examples.get_path('GData_utm.csv'))


y = np.array(data.by_col['PctBach']).reshape((-1,1))
#pov = np.array(data.by_col['PctPov']).reshape((-1,1))
rural = np.array(data.by_col['PctRural']).reshape((-1,1))
#black = np.array(data.by_col['PctBlack']).reshape((-1,1))
fb = np.array(data.by_col['PctFB']).reshape((-1,1))

X = np.hstack([fb, rural])

#Coordinates for calibration points
u = data.by_col['X']
v = data.by_col['Y']
coords = zip(u,v)

In [241]:
smf.OLS(y, add_constant(X)).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.542
Model:                            OLS   Adj. R-squared:                  0.536
Method:                 Least Squares   F-statistic:                     92.19
Date:                Tue, 20 Mar 2018   Prob (F-statistic):           3.71e-27
Time:                        12:42:25   Log-Likelihood:                -439.73
No. Observations:                 159   AIC:                             885.5
Df Residuals:                     156   BIC:                             894.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.7589      1.214     11.336      0.000      11.361      16.156
x1             2.1927      0.298      7.350      0.000       1.603       2.782
x2            -0.0754      0.014     -5.541      0.000      -0.102      -0.049
==============================================================================
Omnibus:                       55.516   Durbin-Watson:                   1.932
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              242.124
Skew:                           1.215   Prob(JB):                     2.65e-53
Kurtosis:                       8.536   Cond. No.                         301.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [242]:

sel = Sel_BW(coords, y, X)

bw = sel.search()
print bw
results = GWR(coords, y, X, bw).fit()

116.0


In [243]:
results.params[0:5]

array([[14.89171769,  1.00531085, -0.09052556],
       [14.90243157,  0.68375976, -0.08828597],
       [14.95911994,  0.88789566, -0.09068385],
       [14.36068699,  0.64815011, -0.0756069 ],
       [11.18322329,  3.53249027, -0.04908478]])

In [244]:
selector = Sel_BW(coords, y, X, multi=True)
bw = selector.search(bw_min=116, bw_max=116, tol_multi=.00001)
err = selector.err
XB = selector.XB
gwr = MGWR(coords, y, X, bw, XB, err).fit()

In [245]:
print gwr.model.bws

[116.0, 116.0, 116.0]


In [246]:
gwr.params[0:5]

array([[12.69228613,  1.44645508, -0.06932414],
       [12.66488349,  1.15750432, -0.06858848],
       [12.6869407 ,  1.35180529, -0.06917044],
       [12.68413603,  1.05254214, -0.06386853],
       [13.00327741,  3.07701635, -0.06428163]])

In [247]:
#MGWR hat matrix
S = selector.bw[-1]

#lets manually compute sigma**2. We can define it based on H+T or as that from GWR
sigma1 = gwr.resid_ss / np.trace(np.dot(S, S.T))
sigma2 = gwr.resid_ss / (len(gwr.model.y) - (2*np.trace(S)) + np.trace(np.dot(S.T, S)))

In [248]:
#GWR vs MGWR for sigma**2
results.sigma2_v1v2, sigma1, sigma2

(11.565017720788457, 287.40196998705403, 11.611276586360756)

In [249]:
#GWR standard errors
results.bse[0:5]

array([[1.51548627, 0.51691193, 0.01729984],
       [1.39609778, 0.41515309, 0.01635002],
       [1.46518838, 0.48102361, 0.0168942 ],
       [1.51082863, 0.38324097, 0.01782204],
       [1.70691315, 0.39529175, 0.01921924]])

In [250]:
#MGWR standard errors according to H+T
S_0 = gwr.S[0]
S_1 = gwr.S[1]
S_2 = gwr.S[2]
bse0 = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma1)).reshape((-1,1))
bse1 = np.sqrt(np.diag(np.dot(S_1, S_1.T)*sigma1)).reshape((-1,1))
bse2 = np.sqrt(np.diag(np.dot(S_2, S_2.T)*sigma1)).reshape((-1,1))


bse = np.hstack([bse0, bse1, bse2])

bse[0:5]

array([[1.96407491, 1.00514557, 2.02429005],
       [1.90732819, 2.1892616 , 2.5847536 ],
       [1.93549612, 0.40664811, 1.62757605],
       [2.02879045, 0.14258271, 2.74253499],
       [1.97758725, 1.57784313, 1.14208746]])

In [251]:
#MGWR standard errors according to H+T but using GWR type DOF
S_0 = gwr.S[0]
S_1 = gwr.S[1]
S_2 = gwr.S[2]
bse0 = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma2)).reshape((-1,1))
bse1 = np.sqrt(np.diag(np.dot(S_1, S_1.T)*sigma2)).reshape((-1,1))
bse2 = np.sqrt(np.diag(np.dot(S_2, S_2.T)*sigma2)).reshape((-1,1))


bse = np.hstack([bse0, bse1, bse2])

bse[0:5]

array([[0.3947782 , 0.20203382, 0.40688142],
       [0.38337213, 0.44004063, 0.51953435],
       [0.38903388, 0.08173609, 0.32714208],
       [0.407786  , 0.02865906, 0.55124834],
       [0.39749417, 0.31714578, 0.22955908]])

In [252]:
#MGWR standard errors according to H+T and using Hanchen's suggestion to factor out Xj
X_0 = gwr.model.X[:,0]
S_0 = gwr.S[0]
S_0 = np.linalg.inv(np.diag(X_0)) * S_0
X_1 = gwr.model.X[:,1]
S_1 = gwr.S[1]
S_1 = np.linalg.inv(np.diag(X_1)) * S_1
X_2 = gwr.model.X[:,2]
S_2 = gwr.S[2]
S_2 = np.linalg.inv(np.diag(X_2)) * S_2

bse0 = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma1)).reshape((-1,1))
bse1 = np.sqrt(np.diag(np.dot(S_1, S_1.T)*sigma1)).reshape((-1,1))
bse2 = np.sqrt(np.diag(np.dot(S_2, S_2.T)*sigma1)).reshape((-1,1))


bse = np.hstack([bse0, bse1, bse2])

bse[0:5]

array([[0.33424266, 0.14081507, 0.00469399],
       [0.31786451, 0.2903543 , 0.00595136],
       [0.32419619, 0.05568167, 0.00372938],
       [0.35370109, 0.01506339, 0.00653359],
       [0.37472465, 0.2144367 , 0.00281555]])

In [253]:
#MGWR standard errors according to H+T but using GWR type DOF and using Hanchen's suggestion to factor out Xj
X_0 = gwr.model.X[:,0]
S_0 = gwr.S[0]
S_0 = np.linalg.inv(np.diag(X_0)) * S_0
X_1 = gwr.model.X[:,1]
S_1 = gwr.S[1]
S_1 = np.linalg.inv(np.diag(X_1)) * S_1
X_2 = gwr.model.X[:,2]
S_2 = gwr.S[2]
S_2 = np.linalg.inv(np.diag(X_2)) * S_2

bse0 = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma2)).reshape((-1,1))
bse1 = np.sqrt(np.diag(np.dot(S_1, S_1.T)*sigma2)).reshape((-1,1))
bse2 = np.sqrt(np.diag(np.dot(S_2, S_2.T)*sigma2)).reshape((-1,1))


bse = np.hstack([bse0, bse1, bse2])

bse[0:5]

array([[0.06718263, 0.02830377, 0.00094349],
       [0.06389063, 0.05836109, 0.00119622],
       [0.06516329, 0.01119199, 0.0007496 ],
       [0.07109376, 0.00302774, 0.00131325],
       [0.07531949, 0.04310168, 0.00056593]])

In [254]:
#MGWR standard errors if we just pull out the individual GWR standard error for the single component
np.hstack([gwr.SE[0], gwr.SE[1], gwr.SE[2]])[0:5]

array([[0.38743249, 0.30962013, 0.00528237],
       [0.37623866, 0.27316231, 0.00509914],
       [0.38179505, 0.29691709, 0.00520396],
       [0.40019825, 0.25553735, 0.00541041],
       [0.39009793, 0.2175244 , 0.00527654]])

In [255]:
#Finally, lets try this with a constant but with standardized variables

In [256]:
data = ps.open(ps.examples.get_path('GData_utm.csv'))


y = np.array(data.by_col['PctBach']).reshape((-1,1))
#pov = np.array(data.by_col['PctPov']).reshape((-1,1))
rural = np.array(data.by_col['PctRural']).reshape((-1,1))
#black = np.array(data.by_col['PctBlack']).reshape((-1,1))
fb = np.array(data.by_col['PctFB']).reshape((-1,1))

X = np.hstack([fb, rural])

#Coordinates for calibration points
u = data.by_col['X']
v = data.by_col['Y']
coords = zip(u,v)

X = (X - X.mean()) / X.std()
y = (y - y.mean()) / y.std()

In [257]:
smf.OLS(y, add_constant(X)).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.542
Model:                            OLS   Adj. R-squared:                  0.536
Method:                 Least Squares   F-statistic:                     92.19
Date:                Tue, 20 Mar 2018   Prob (F-statistic):           3.71e-27
Time:                        12:42:32   Log-Likelihood:                -163.58
No. Observations:                 159   AIC:                             333.2
Df Residuals:                     156   BIC:                             342.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.7881      1.770      7.788      0.000      10.291      17.285
x1            15.2377      2.073      7.350      0.000      11.143      19.333
x2            -0.5240      0.095     -5.541      0.000      -0.711      -0.337
==============================================================================
Omnibus:                       55.516   Durbin-Watson:                   1.932
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              242.124
Skew:                           1.215   Prob(JB):                     2.65e-53
Kurtosis:                       8.536   Cond. No.                         82.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [258]:

sel = Sel_BW(coords, y, X)

bw = sel.search()
print bw
results = GWR(coords, y, X, bw).fit()

116.0


In [259]:
results.params[0:5]

array([[ 6.43794765,  6.98627247, -0.6290952 ],
       [ 4.43507389,  4.75169646, -0.61353149],
       [ 5.71164464,  6.17031141, -0.63019523],
       [ 4.19571403,  4.50423195, -0.52541996],
       [21.91172376, 24.54856585, -0.34110805]])

In [260]:
selector = Sel_BW(coords, y, X, multi=True)
bw = selector.search(bw_min=116, bw_max=116, tol_multi=.00001)
err = selector.err
XB = selector.XB
gwr = MGWR(coords, y, X, bw, XB, err).fit()

In [261]:
print gwr.model.bws

[116.0, 116.0, 116.0]


In [262]:
gwr.params[0:5]

array([[14.94031916, 16.77654707, -0.5461347 ],
       [14.91127895, 16.80255771, -0.53373777],
       [14.93091434, 16.78495429, -0.54377117],
       [14.90128811, 16.80653358, -0.45013835],
       [15.11982444, 16.60979377, -0.5242825 ]])

In [263]:
#MGWR hat matrix
S = selector.bw[-1]

#lets manually compute sigma**2. We can define it based on H+T or as that from GWR
sigma1 = gwr.resid_ss / np.trace(np.dot(S, S.T))
sigma2 = gwr.resid_ss / (len(gwr.model.y) - (2*np.trace(S)) + np.trace(np.dot(S.T, S)))

In [264]:
#GWR vs MGWR for sigma**2
results.sigma2_v1v2, sigma1, sigma2

(0.35858238770583906, 11.023087731791867, 0.42722706487641793)

In [265]:
#GWR standard errors
results.bse[0:5]

array([[3.11948908, 3.59220989, 0.12022288],
       [2.50026944, 2.88505052, 0.11362229],
       [2.90158004, 3.34280886, 0.11740394],
       [2.29309935, 2.66328155, 0.12385184],
       [2.3275286 , 2.74702682, 0.13356154]])

In [266]:
#MGWR standard errors according to H+T
S_0 = gwr.S[0]
S_1 = gwr.S[1]
S_2 = gwr.S[2]
bse0 = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma1)).reshape((-1,1))
bse1 = np.sqrt(np.diag(np.dot(S_1, S_1.T)*sigma1)).reshape((-1,1))
bse2 = np.sqrt(np.diag(np.dot(S_2, S_2.T)*sigma1)).reshape((-1,1))


bse = np.hstack([bse0, bse1, bse2])

bse[0:5]

array([[0.38464918, 0.38768027, 0.36856511],
       [0.37353576, 0.36660641, 0.57033735],
       [0.37905224, 0.3861454 , 0.23662455],
       [0.39732323, 0.40760734, 0.60419175],
       [0.38729547, 0.38381436, 0.06335296]])

In [267]:
#MGWR standard errors according to H+T but using GWR type DOF
S_0 = gwr.S[0]
S_1 = gwr.S[1]
S_2 = gwr.S[2]
bse0 = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma2)).reshape((-1,1))
bse1 = np.sqrt(np.diag(np.dot(S_1, S_1.T)*sigma2)).reshape((-1,1))
bse2 = np.sqrt(np.diag(np.dot(S_2, S_2.T)*sigma2)).reshape((-1,1))


bse = np.hstack([bse0, bse1, bse2])

bse[0:5]

array([[0.07572557, 0.0763223 , 0.07255911],
       [0.07353768, 0.07217351, 0.11228185],
       [0.07462371, 0.07602013, 0.04658408],
       [0.0782207 , 0.08024533, 0.11894674],
       [0.07624654, 0.07556122, 0.01247225]])

In [268]:
#MGWR standard errors according to H+T and using Hanchen's suggestion to factor out Xj
X_0 = gwr.model.X[:,0]
S_0 = gwr.S[0]
S_0 = np.linalg.inv(np.diag(X_0)) * S_0
X_1 = gwr.model.X[:,1]
S_1 = gwr.S[1]
S_1 = np.linalg.inv(np.diag(X_1)) * S_1
X_2 = gwr.model.X[:,2]
S_2 = gwr.S[2]
S_2 = np.linalg.inv(np.diag(X_2)) * S_2

bse0 = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma1)).reshape((-1,1))
bse1 = np.sqrt(np.diag(np.dot(S_1, S_1.T)*sigma1)).reshape((-1,1))
bse2 = np.sqrt(np.diag(np.dot(S_2, S_2.T)*sigma1)).reshape((-1,1))


bse = np.hstack([bse0, bse1, bse2])

bse[0:5]

array([[0.06545889, 0.07491731, 0.0601281 ],
       [0.06225136, 0.06944765, 0.09379481],
       [0.06349137, 0.07347428, 0.03831354],
       [0.06926968, 0.08095787, 0.09967733],
       [0.07338698, 0.0826819 , 0.01052659]])

In [269]:
#MGWR standard errors according to H+T but using GWR type DOF and using Hanchen's suggestion to factor out Xj
X_0 = gwr.model.X[:,0]
S_0 = gwr.S[0]
S_0 = np.linalg.inv(np.diag(X_0)) * S_0
X_1 = gwr.model.X[:,1]
S_1 = gwr.S[1]
S_1 = np.linalg.inv(np.diag(X_1)) * S_1
X_2 = gwr.model.X[:,2]
S_2 = gwr.S[2]
S_2 = np.linalg.inv(np.diag(X_2)) * S_2

bse0 = np.sqrt(np.diag(np.dot(S_0, S_0.T)*sigma2)).reshape((-1,1))
bse1 = np.sqrt(np.diag(np.dot(S_1, S_1.T)*sigma2)).reshape((-1,1))
bse2 = np.sqrt(np.diag(np.dot(S_2, S_2.T)*sigma2)).reshape((-1,1))


bse = np.hstack([bse0, bse1, bse2])

bse[0:5]

array([[0.01288684, 0.01474891, 0.01183737],
       [0.01225537, 0.0136721 , 0.01846531],
       [0.01249949, 0.01446482, 0.00754276],
       [0.01363706, 0.01593811, 0.01962339],
       [0.01444764, 0.01627752, 0.00207236]])

In [271]:
#MGWR standard errors if we just pull out the individual GWR standard error for the single component
np.hstack([gwr.SE[0], gwr.SE[1], gwr.SE[2]])[0:5]

array([[0.07451606, 0.08464774, 0.07055087],
       [0.07236312, 0.08225453, 0.06777438],
       [0.0734318 , 0.08343102, 0.06946867],
       [0.07697134, 0.08767169, 0.07179737],
       [0.07502872, 0.08573802, 0.06876415]])